In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import re

In [2]:
import numpy as np 
import pandas as pd

In [3]:
codingProfileData = pd.read_csv("coding-profile-data.csv")
codingProfileData

,Email,Name,Codechef,Codeforces,Spoj,Leetcode,Gfg
0,sanchitamishra170676@gmail.com,sanchita,sanchita170676,sanchita170676,sanchita170676,sanchitamishra170676,sanchitamishra170676
1,hemkishan12@gmail.com,Hem Kishan,vyas_7275,hem_7275,hem_7275,hem_7275,hem_7275
2,hanu.2024cs1020@kiet.edu,Hanu agarwal,hanu_30,NaN,NaN,hanuagarwal181,hanu_30
3,ashibh21@gmail.com,Ashish bhatt,ashb21,ashibh,NaN,ashishbhatt,ashibh21
4,shivam.1923ec1084@kiet.edu,Shivam Sharma,shiv0618,shiv3116,NaN,shiv3116,shivashar132001
5,vaishnavi.2024cs1029@kiet.edu,Vaishnavi Sangal,vaishnavi_2308,NaN,NaN,Break_The_Code,vaishnavisangal787
6,garvit.2125csme1002@kiet.edu,Garvit singhal,garvit_47,garvit_singhal,Garvit_singhal,Garvit_singhal,garvitsinghal47
7,aditya.2125cse1127@kiet.edu,Aditya Agnihotri,theadi,theadityaagnihotri,NaN,theadityaagnihotri,theadityaagnihotri
8,garvit.2125cse1150@kiet.edu,Garvit Agarwal,garvit414,garvit414,NaN,garvit414,NaN
9,shubham.2125cse1159@kiet.edu,Shubham Verma,shubh50,shubhambbk50,NaN,shubhambbk50,NaN


In [4]:
profileStatsDataFrame = pd.DataFrame()

In [5]:
class ProfileScraper():
    def __init__(self, name, email, codechef, codeforces, spoj, leetcode, gfg):
#         launch browser in headless state
        options = Options()
        options.add_argument('--headless')
        self.driver = webdriver.Chrome(options=options)
        
        global profileStatsDataFrame
        profileStats = {"name": name, "email": email}
        
        self.profiles = {
            "codechef": {f"https://www.codechef.com/users/{codechef}": "//h5[contains(text(), 'Fully Solved')]"},
            "codeforces": {f"https://codeforces.com/profile/{codeforces}": "//div[contains(@class, '_UserActivityFrame_counterValue')]"},
            "spoj": {f"https://www.spoj.com/users/{spoj}/": "//dl[contains(@class, 'profile-info-data')]/dd"},
            "leetcode": {f"https://leetcode.com/{leetcode}/": "//div[contains(@class, 'font-medium text-label-1')]"},
            "gfg": {f"https://auth.geeksforgeeks.org/user/{gfg}/practice": "//span[contains(@class, 'score_card_value')]"},
        }
        
        totalQuestions = self.getProfileStats(profileStats)
        profileStats["total"] = totalQuestions
#         adding individual's data to global profile stats data frame
        individualProfileStatsDataFrame = pd.DataFrame([profileStats])
        profileStatsDataFrame = pd.concat([profileStatsDataFrame, individualProfileStatsDataFrame], ignore_index=True)
        
    
    def getProfileStats(self, profileStats):
        '''function to scrape the data of user on different platforms, 
        adds the data in profile stats dictionary and returns total questions solved'''
        totalQuestions = 0
        
        for platform, data in self.profiles.items():
            for profile, query in data.items():
                questions = self.runQuery(profile, query)
#                 print(profile, " ", questions, end="\n")
                profileStats.update({platform:questions})
                totalQuestions += int(questions)
        
        return totalQuestions
    
    
    def runQuery(self, profile, query):
        try:
            self.driver.get(profile)
            element = WebDriverWait(self.driver, 5).until(
                EC.presence_of_element_located((By.XPATH, query))
            )
            questions = int(re.search(r'\d+', element.text).group())
            return questions
        except:
            return '0'
        


In [6]:
# Iterating through ethe profile data of all individuals
for ind in codingProfileData.index:
    ProfileScraper(codingProfileData.loc[ind,"Email"], codingProfileData.loc[ind,"Name"],codingProfileData.loc[ind,"Codechef"],codingProfileData.loc[ind,"Codeforces"],codingProfileData.loc[ind,"Spoj"],codingProfileData.loc[ind,"Leetcode"],codingProfileData.loc[ind,"Gfg"])
    print(codingProfileData.loc[ind,"Name"])

sanchita
Hem Kishan
Hanu agarwal
Ashish bhatt
Shivam Sharma
Vaishnavi Sangal
Garvit singhal
Aditya Agnihotri
Garvit Agarwal
Shubham Verma
Vaibhav Bisht
Jayant Kumar Gupta 
Gaurav Kumar
Ali Haider
Priyanshu Agarwal
Abhinav
Shivam Sharma
Samyak Jain
Yashveer Singh
Mukul Dixit
Sahil Jain 
Paras Tiwari
Ibtesam Umar Ansari
Abhay Singh
Ankita Jain 
Shifa Rifat
Anukool Kesharwani
Aman Yadav 
Aditya Kumar 
Anugam Jain
Naman Dwivedi


In [7]:
# Data set collected after scrapping
profileStatsDataFrame

,name,email,codechef,codeforces,spoj,leetcode,gfg,total
0,sanchitamishra170676@gmail.com,sanchita,131,38,0,136,127,432
1,hemkishan12@gmail.com,Hem Kishan,12,374,0,271,345,1002
2,hanu.2024cs1020@kiet.edu,Hanu agarwal,61,423,0,0,2456,2940
3,ashibh21@gmail.com,Ashish bhatt,143,370,0,0,104,617
4,shivam.1923ec1084@kiet.edu,Shivam Sharma,147,152,0,419,293,1011
5,vaishnavi.2024cs1029@kiet.edu,Vaishnavi Sangal,132,423,0,565,774,1894
6,garvit.2125csme1002@kiet.edu,Garvit singhal,48,1,0,388,91,528
7,aditya.2125cse1127@kiet.edu,Aditya Agnihotri,16,190,0,3,42,251
8,garvit.2125cse1150@kiet.edu,Garvit Agarwal,101,11,0,35,0,147
9,shubham.2125cse1159@kiet.edu,Shubham Verma,45,67,0,82,0,194


In [10]:
profileStatsDataFrame.to_csv("profile-staistics.csv", index = False)